In [1]:
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup

### 1) Varrer os textos do Arquivo Pessoa

Fomos buscar os textos ao [Arquivo Pessoa](http://arquivopessoa.net/). Para isso, usámos o [Wget](https://www.gnu.org/software/wget/manual/wget.html).

Para ir buscar as páginas de todos os textos disponíveis no site, basta colocar na linha de comandos:

``` for i in {0..9999}; do wget http://arquivopessoa.net/textos/$i; sleep 1; done ```

_Como estamos a fazer um pedido por segundo, este processo deverá levar quase três horas a transferir todos os documentos._

### 2) Limpar os ficheiros HTML

Depois de colocarmos todos os ficheiros HTML na mesma pasta, vamos iterá-los para removermos as tags.

In [2]:
#ir buscar os ficheiros html
file_location = os.path.join('pessoa-textos', '*.html')

#inicialização de lista vazia; a info de cada html é convertida num dicionário
dict_lst = []

for file in glob.glob(file_location):
    with open(file) as fp:
        
        soup = BeautifulSoup(fp)
        
        #ir buscar autor
        autor = soup.find("div", attrs = {"class": "autor"}).string
            
        #ir buscar título + limpar título
        titulo = soup.find("h1", attrs = {"class": "titulo-texto"})
        titulo_limpo = titulo.text.replace('\n', '').replace('\xa0', '')  
            
        #ir buscar data
        data = soup.find("div", attrs = {"class": "data"}).string 
            
        try:
            #ir buscar poema + limpar poema
            texto = soup.find("div", attrs = {"class": "texto-poesia"})
            texto_limpo = texto.text.replace('<p>', '').replace('\xa0', '').replace('</p>','\n')
            poema_dict = {}
            for variable in ["autor", "titulo_limpo", "texto_limpo", "data"]:
                poema_dict[variable] = eval(variable)
            poema_dict["text_type"] = "poesia"
            dict_lst.append(poema_dict)
        
        except:
            #ir buscar prosa + limpar prosa
            texto = soup.find("div", attrs = {"class": "texto-prosa"})
            texto_limpo = texto.text.replace('<p>', '').replace('\xa0', '').replace('</p>','\n')       
            prosa_dict = {}
            for variable in ["autor", "titulo_limpo", "texto_limpo", "data"]:
                prosa_dict[variable] = eval(variable)
            prosa_dict["text_type"] = "prosa"
            dict_lst.append(prosa_dict)
            
    #criar o conjunto de dados     
    full_data = pd.DataFrame(dict_lst)

In [3]:
#ajustar nomes de colunas

full_data.rename(columns={"titulo_limpo":"título","texto_limpo":"texto", "text_type":"tipo"}, inplace=True)

#remover quebras de linhas no início dos textos
full_data["texto"] = full_data.texto.str.lstrip()

#exportar para csv
full_data.to_csv("textos-pessoa.csv", index=False)